# Notebook to plot the distribution of the caudal-rostral length of the spinal levels

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
# Path to csv files with information about the spinal levels
PATH_pred = "/Users/theomathieu/Downloads/fullD3_pred/res/dist_pred.csv"

In [3]:
df_pred = pd.read_csv(PATH_pred)
display(df_pred.head())

,level,sub_name,spinal_start,spinal_end,height,vertebrae_start,vertebrae_end
0,2,sub-mountSinai02_103_0000,208.0,219.0,8.8,0,1
1,3,sub-mountSinai02_103_0000,184.0,203.0,15.2,0,1
2,4,sub-mountSinai02_103_0000,166.0,179.0,10.4,0,1
3,5,sub-mountSinai02_103_0000,144.0,155.0,8.8,0,1
4,6,sub-mountSinai02_103_0000,123.0,136.0,10.4,0,1


In [4]:
# Add z-score to find outliers
z_scores = (df_pred['height'] - df_pred['height'].mean()) / df_pred['height'].std()
df_pred["z_scores"] = z_scores

### Dataframe creation

In [5]:
df_height = df_pred[["height", "level"]].copy()
df_height["mesure"] = "length"
df_height["from"] = "pred_value"
df_height.rename(columns={"height": "value"}, inplace=True)
display(df_height.head())

,value,level,mesure,from
0,8.8,2,length,pred_value
1,15.2,3,length,pred_value
2,10.4,4,length,pred_value
3,8.8,5,length,pred_value
4,10.4,6,length,pred_value


In [6]:
for level in df_height["level"].unique():
    new_line_mean = pd.DataFrame({"value": [df_height.loc[df_height["level"] == level, "value"].mean()],
                                  "level": [level.astype(int)],
                                  "mesure": ["mean"],
                                  "from": ["pred"]})
    df_height = pd.concat([df_height, new_line_mean], ignore_index=True)
    new_line_std = pd.DataFrame({"value": [df_height.loc[df_height["level"] == level, "value"].std()],
                                 "level": [level.astype(int)],
                                 "mesure": ["std"],
                                 "from": ["pred"]})
    df_height = pd.concat([df_height, new_line_std], ignore_index=True)
display(df_height.head())

,value,level,mesure,from
0,8.8,2,length,pred_value
1,15.2,3,length,pred_value
2,10.4,4,length,pred_value
3,8.8,5,length,pred_value
4,10.4,6,length,pred_value


In [7]:
cadotte = {
    '3': {'mean': 10.5, 'std': 2.2},
    '4': {'mean': 9.9, 'std': 1.3},
    '5': {'mean': 10.5, 'std': 1.5},
    '6': {'mean': 9.7, 'std': 1.6},
    '7': {'mean': 9.4, 'std': 1.4},
    '8': {'mean': 9.6, 'std': 1.4},
}
df_cadotte_mean = pd.DataFrame(cadotte).T[['mean']]
df_cadotte_mean["level"] = df_cadotte_mean.index.astype(int)
df_cadotte_mean["from"] = "cadotte"
df_cadotte_mean["mesure"] = "mean"
df_cadotte_mean.rename(columns={"mean": "value"}, inplace=True)
df_cadotte_std = pd.DataFrame(cadotte).T[['std']]
df_cadotte_std["level"] = df_cadotte_mean.index.astype(int)
df_cadotte_std["from"] = "cadotte"
df_cadotte_std["mesure"] = "std"
df_cadotte_std.rename(columns={"std": "value"}, inplace=True)
df_cadotte = pd.concat([df_cadotte_mean, df_cadotte_std], ignore_index=True)
display(df_cadotte.head())
df = pd.concat([df_height, df_cadotte], ignore_index=True)
display(df.head())
df_mean_std = df[df["mesure"] != "length"]
pivot_df = df_mean_std.pivot_table(index=['level', 'from'], columns='mesure', values='value', aggfunc='first')

# Reset the index
pivot_df = pivot_df.reset_index()
pivot_df['level_offset'] = pivot_df.apply(
    lambda row: row['level'] - 0.1 if row['from'] == 'pred' else row['level'] + 0.1, axis=1)
display(pivot_df.head())

,value,level,from,mesure
0,10.5,3,cadotte,mean
1,9.9,4,cadotte,mean
2,10.5,5,cadotte,mean
3,9.7,6,cadotte,mean
4,9.4,7,cadotte,mean


,value,level,mesure,from
0,8.8,2,length,pred_value
1,15.2,3,length,pred_value
2,10.4,4,length,pred_value
3,8.8,5,length,pred_value
4,10.4,6,length,pred_value


mesure,level,from,mean,std,level_offset
0,2,pred,5.430159,2.128017,1.9
1,3,cadotte,10.500000,2.200000,3.1
2,3,pred,10.422222,2.242638,2.9
3,4,cadotte,9.900000,1.300000,4.1
4,4,pred,6.953448,2.296077,3.9


In [30]:
fig = px.scatter(pivot_df, x="level_offset", y="mean", color="from", error_y="std")
fig.update_layout(
    title="Caudal-rostral length of the spinal levels",
    xaxis_title="Spinal level",
    yaxis_title="Caudal-rostral length (mm)",
    xaxis=dict(ticktext=["C" + x.astype(str) for x in df["level"].unique()], tickvals=df["level"].unique())
)
fig.show()

In [23]:
fig = px.scatter(pivot_df, x="level_offset", y="mean", color="from", error_y="std")
for level in df["level"].unique():
    df_level = df[df["level"] == level]
    mean = df_level.loc[df_level["mesure"] == "mean"]
    std = df_level.loc[df_level["mesure"] == "std"]
    #TODO handle nan values
    scatter = px.scatter(df_level, x="level", y="value").data[0]
    fig.add_trace(scatter)

fig.show()

## PMJ DISTANCE

In [63]:
PATH_pmj = "/Users/theomathieu/Downloads/indiv.csv"
df_pmj = pd.read_csv(PATH_pmj)
display(df_pmj.head())
PATH_cad = "/Users/theomathieu/Downloads/cadot.csv"
df_cad = pd.read_csv(PATH_cad)
display(df_cad.head())

,level,sub_name,spinal_start,spinal_end,height,PMJ_start,PMJ_end
0,2,sub-mountSinai02_103_0000,208.0,219.0,8.8,-40,-58
1,3,sub-mountSinai02_103_0000,184.0,203.0,15.2,-65,-75
2,4,sub-mountSinai02_103_0000,166.0,179.0,10.4,-81,-94
3,5,sub-mountSinai02_103_0000,144.0,155.0,8.8,-96,-105
4,6,sub-mountSinai02_103_0000,123.0,136.0,10.4,-110,-118


,level,sub_name,spinal_start,spinal_end,height,PMJ_start,PMJ_end
0,2,sub-mountSinai02_103_0000,208.0,219.0,8.8,NaN,NaN
1,3,sub-mountSinai02_103_0000,184.0,203.0,15.2,-65.0,-75.0
2,4,sub-mountSinai02_103_0000,166.0,179.0,10.4,-81.0,-94.0
3,5,sub-mountSinai02_103_0000,144.0,155.0,8.8,-96.0,-105.0
4,6,sub-mountSinai02_103_0000,123.0,136.0,10.4,-110.0,-118.0


In [74]:
subject = 1
colors = ['#5B6C5D', '#5CC878', '#FF707D', '#64DEDC', '#D682C6', '#E1E074','#738282', '#C16200', '#197278', '#EF9CDA']
fig = go.Figure()
for idx, level in enumerate(df_pmj["level"].unique()):
    df_level_pmj = df_pmj[df_pmj["level"] == level]
    df_level_cad = df_cad[df_cad["level"] == level]
    if not np.isnan(df_level_pmj["PMJ_start"].values[0]):
        fig.add_shape(
            type="rect",
            x0=subject-0.3, y0=df_level_pmj["PMJ_start"].values[0], x1=subject-0.05, y1=df_level_pmj["PMJ_end"].values[0],
            line=dict(color=colors[idx], width=2),
            fillcolor=colors[idx],
            name=f"Predicted {level}"
        )
    if not np.isnan(df_level_cad["PMJ_start"].values[0]):
        fig.add_shape(
            type="rect",
            x0=subject+0.05, y0=df_level_cad["PMJ_start"].values[0], x1=subject+0.3, y1=df_level_cad["PMJ_end"].values[0],
            line=dict(color=colors[idx], width=3),
            name=f"Cadotte {level}"
        )
fig.update_layout(
    title="Nerve rootlet layout",
    xaxis_title="Subject",
    yaxis_title="PMJ distance (PMJ = 0 mm)",
    xaxis_range=[0, 10],  # Adjust the x-axis range as needed
    yaxis_range=[min(df_pmj["PMJ_end"])-25, 2],  # Adjust the y-axis range as needed
    yaxis_autorange='reversed',
    showlegend=True,
    xaxis=dict(ticktext=[str(x) for x in range(1,10)], tickvals=[*range(1,10)])
)
fig.show()